## Konduit Serving BMI Calculator Setup
Body mass index (BMI) is a measure of body fat based on height and weight. The standard way of calculating BMI is through the formula below:

$$ \text{BMI}=\frac{m} {{h}^2} $$

$$ \text{BMI} = \text{Body Mass Index} $$

$$ \text{m} = \text{mass (in kilograms/pounds)} $$

$$ \text{h} = \text{height (in meters/feet)} $$

### A new approach for estimating Body Mass Index using facial features with Konduit-Serving
Measuring weight and height of individual people is time-consuming while also being error prone. In this notebook, we'll see how we can measure BMI of an individual just by looking at their facial features. The backend server used for taking image data and providing BMI values is served using Konduit-Serving, which is a high performance model pipeline server. Model training, gathering and preparing dataset is out of the scope of this notebook. The main workflow we'll look at is how to serve a model that can look at a person's face and answer back with a BMI value through REST API. We'll also be setting up a web server through konduit-serving "custom endpoints" that will make use of a webcam and label the canvas with the detected face along with the corresponding estimated BMI value. 

So, let's get started!  

## Konduit Serving Components

Before diving deep into the coding part, let's look at some of the few konduit-serving components in detail. 

### 1. CLI Interface
Konduit-Serving comes with a CLI interface (with a `konduit` alias) that's responsible of taking care of most aspects of the application. The help command will describe most of what we are able to do with konduit-serving. Executing `konduit --help` command will show us the following output:

```bash
$ konduit --help
-------------------------------------------------------------------------------------------------
Usage: konduit [COMMAND] [OPTIONS] [arg...]

Commands:
    build         Command line interface for performing Konduit Serving builds.
    config        A helper command for creating boiler plate json/yaml for
                  inference configuration
    inspect       Inspect the details of a particular konduit server.
    list          Lists the running konduit servers.
    logs          View the logs of a particular konduit server
    predict       Run inference on konduit servers using given inputs
    profile       Command to List, view, edit, create and delete konduit serving
                  run profiles.
    pythonpaths   A utility command to manage system installed and manually
                  registered python binaries.
    serve         Start a konduit server application
    stop          Stop a running konduit server
    version       Displays konduit-serving version.

Run 'konduit COMMAND --help' for more information on a command.
-------------------------------------------------------------------------------------------------
```

Each command describes its short hand description right in front of it. If you want to look at an individual command in detail, you can use the corresponding `--help` command with them. For example, the help menu for the `logs` command can be seen by executing, `konduit logs --help`:

```bash
$ konduit logs --help
-------------------------------------------------------------------------------------------------
Usage: konduit logs  [-f] [-l <value>]  server-id

View the logs of a particular konduit server

View the logs of a particular konduit server given an id.

Example usages:
--------------
- Outputs the log file contents of server with an id of 'inf_server':
$ konduit logs inf_server

- Outputs and tail the log file contents of server with an id of 'inf_server':
$ konduit logs inf_server -f

- Outputs and tail the log file contents of server with an id of 'inf_server'
  from the last 10 lines:
$ konduit logs inf_server -l 10 -f
--------------

Options and Arguments:
 -f,--follow          Follow the logs output.
 -l,--lines <value>   Sets the number of lines to be printed. Default is '10'.
                      Use -1 for outputting everything.

 <server-id>          Konduit server id
-------------------------------------------------------------------------------------------------
```

As you can see, the `--help` command for an individual help command describes its functionality in detail along with some explicit examples and use cases. It also describes each individual optional/non-optional argument that can be used with it. This can come in very handy while learning about konduit-serving for the first time and is a useful starting place to play around with a specific command. You can do the same for the rest of the commands. Which are: `build, config, inspect, list, logs, predict, profile, pythonpaths, serve, stop, version`

### 2. Konduit-Serving JAR file
Each konduit-serving distribution whether it is for Windows, Linux or MacOS comes contained in a JAR file. So, you'll need a Java Virtual Machine present in the system where you're using Konduit-Serving as a Model Pipeline Server. The CLI itself is linked with the jar file and utilizes a java runtime internally to interact with the Konduit-Serving package. If you look at the konduit serving distribution, you'll see the following folder architecture in the root folder:

konduit (root folder) \
├── **bin** (directory) \
&nbsp;&nbsp;&nbsp;&nbsp;└── **konduit** (konduit serving CLI script) \
├── **conf** (directory) \
&nbsp;&nbsp;&nbsp;&nbsp;└── **konduit-serving-env.sh** (for configuration of different environment variables for konduit-serving) \
└── **konduit.jar** (main package JAR file) 

The main CLI logic is places under `bin/konduit` file. If you see the contents of that file, you'll see the following output:
```bash
#!/usr/bin/env bash

SCRIPT_DIR="$(dirname "$0")"

. ${SCRIPT_DIR}/../conf/konduit-serving-env.sh

java -jar -Dvertx.cli.usage.prefix=konduit ${SCRIPT_DIR}/../konduit.jar "$@"
```

As you can see, it uses the `java` command which is available through a Java runtime environment. The java command itself uses the `konduit.jar` file which is the main application package inside a Konduit-Serving distribution.

This JAR file will be uses as a Java application dependency while creating custom endpoints logic for a Konduit-Serving pipeline. We'll get to how we can do that later in this notebook.

### What is a Model Serving Pipeline?
Throughout in this notebook we have used the term "Model Serving Pipeline". This refers to how Machine Learning or Deep Learning models get served on an application server. Machine and Deep Learning models work on n-dimensional arrays (also knows as ND-Arrays). They don't know how to convert a JPEG or PNG image into numbers directly. Instead, they expect pre-processed data in the form of multi-dimensional array. Also, any other form of data, be it text, audio or video, gets converted into numbers ND-Arrays before getting fed into a machine learning model.

The process during which the data is converted from one form to another is called pre-processing and is done just before it's fed as a model input. So, in a sense you can see this as being Lego blocks fitting into each other. One part takes input in a specific form and outputs it into another form, which in turn gets fed into the next part. This chaining of processes creates a series of steps which have specific jobs to perform before the next step and the end result is a machine learning Pipeline. The typical flow of the pipeline looks like the following: 

(1) Input Image in JPEG -> (2) Converts JPEG to ND-Array -> (3) Model -> (4) Output

A pipeline can also be in a form of a directed acyclic graph or DAG where data can flow into the graph and can give multiple outputs. In Konduit-Serving a Pipeline graph can also contain optional graph branches and can also concatenate outputs from multiple graph nodes. For the sake of current goal (BMI Model Serving) we'll stick to a Sequential Pipeline which only has one input and one output.

#### Pipeline Step
Inside Konduit-Serving, a pipeline can be broken down into steps, where each step is responsible for performing a specific function. A pipeline step is the smallest component of a whole pipeline and can be used for a whole list of operations. To see the list of available pipeline steps you can use the `config` command in Konduit-Serving CLI.

```bash
$ konduit config --help
-------------------------------------------------------------------------------------------------
Usage: konduit config  [-m] [-o <output-file>] -p <config> [-pr <value>]  [-y]

A helper command for creating boiler plate json/yaml for inference configuration

This command is a utility to create boilerplate json/yaml configurations that
can be conveniently modified to start konduit servers.

Example usages:
--------------
                     -- FOR SEQUENCE PIPELINES--
- Prints 'logging -> tensorflow -> logging' config in pretty format:
$ konduit config -p logging,tensorflow,logging

- Prints 'logging -> tensorflow -> logging' config with gRPC protocol
  in pretty format:
$ konduit config -p logging,tensorflow,logging -pr grpc

- Prints 'dl4j -> logging' config in minified format:
$ konduit config -p dl4j,logging -m

- Saves 'dl4j -> logging' config in a 'config.json' file:
$ konduit config -p dl4j,logging -o config.json

- Saves 'dl4j -> logging' config in a 'config.yaml' file:
$ konduit config -p dl4j,logging -y -o config.json


                  -- FOR GRAPH PIPELINES --
- Generates a config that logs the input(1) then flow them through two
  tensorflow models(2,3) and merges the output(4):
$ konduit config -p
1=logging(input),2=tensorflow(1),3=tensorflow(1),4=merge(2,3)

- Generates a config that logs the input(1) then channels(2) them through one
  of the two tensorflow models(3,4) and then selects the output(5) based
  on the value of the selection integer field 'select'
$ konduit config -p
1=logging(input),[2_1,2_2]=switch(int,select,1),3=tensorflow(2_1),4=tensorflow(2
_2),5=any(3,4)

- Generates a config that logs the input(1) then channels(2) them through one
  of the two tensorflow models(3,4) and then selects the output(5) based
  on the value of the selection string field 'select' in the selection map
  (x:0,y:1).
$ konduit config -p
1=logging(input),[2_1,2_2]=switch(string,select,x:0,y:1,1),3=tensorflow(2_1),4=t
ensorflow(2_2),5=any(3,4)
--------------

Options and Arguments:
 -m,--minified               If set, the output json will be printed in a single
                             line, without indentations. (Ignored for yaml
                             configuration output)
 -o,--output <output-file>   Optional: If set, the generated json/yaml will be
                             saved here. Otherwise, it's printed on the console.
 -p,--pipeline <config>      A comma-separated list of sequence/graph pipeline
                             steps to create boilerplate configuration from. For
                             sequences, allowed values are: [crop_grid,
                             crop_fixed_grid, dl4j, keras, draw_bounding_box,
                             draw_fixed_grid, draw_grid, draw_segmentation,
                             extract_bounding_box, camera_frame_capture,
                             video_frame_capture, image_to_ndarray, logging,
                             ssd_to_bounding_box, samediff, show_image,
                             tensorflow, nd4jtensorflow, python, onnx]. For
                             graphs, the list item should be in the format
                             '<output>=<type>(<inputs>)' or
                             '[outputs]=switch(<inputs>)' for switches. The
                             pre-defined root input is named, 'input'. Examples
                             are ==> Pipeline step:
                             'a=tensorflow(input),b=dl4j(input)' Merge Step:
                             'c=merge(a,b)' Switch Step (int):
                             '[d1,d2,d3]=switch(int,select,input)' Switch Step
                             (string):
                             '[d1,d2,d3]=switch(string,select,x:1,y:2,z:3,input)
                             'Any Step: 'e=any(d1,d2,d3)' See the examples above
                             for more usage information.
 -pr,--protocol <value>      Protocol to use with the server. Allowed values are
                             [http, grpc, mqtt]
 -y,--yaml                   Set if you want the output to be a yaml
                             configuration.
-------------------------------------------------------------------------------------------------
```


In [77]:
%classpath add jar ../../konduit.jar

In [78]:
%%bash
konduit stop bmi-onnx-pytorch

Stopping konduit server 'bmi-onnx-pytorch'
Application 'bmi-onnx-pytorch' terminated with status 0



In [79]:
package ai.konduit.serving;

import ai.konduit.serving.endpoint.Endpoint;

import io.vertx.core.Handler;
import io.vertx.core.http.HttpMethod;
import io.vertx.ext.web.RoutingContext;

import javax.imageio.ImageIO;
import java.awt.image.BufferedImage;
import java.io.File;
import java.io.IOException;
import java.util.ArrayList;
import java.util.Arrays;
import java.util.List;

import ai.konduit.serving.pipeline.api.pipeline.PipelineExecutor;

import java.util.Timer;
import java.util.TimerTask;
import io.vertx.core.http.HttpHeaders;

import java.io.File;

import java.nio.charset.StandardCharsets;
import org.apache.commons.io.FileUtils;
import io.vertx.core.http.HttpHeaders;
import io.vertx.ext.web.handler.StaticHandler;

import org.slf4j.Logger;
import org.slf4j.LoggerFactory;

public class WebAppEndpoint implements Endpoint {
    
    final static Logger logger = LoggerFactory.getLogger(WebAppEndpoint.class);

    private PipelineExecutor pipelineExecutor;

    public WebAppEndpoint(PipelineExecutor pipelineExecutor) {
        this.pipelineExecutor = pipelineExecutor;
    }

    public HttpMethod type() { return HttpMethod.GET; }

    public String path() { return "/web-app/*"; }

    public List<String> consumes() { return Arrays.asList(); }

    public List<String> produces() { return Arrays.asList("application/html"); }

    @Override
    public Handler<RoutingContext> handler() {
        return handler -> { 
            try {
                logger.info(new File(handler.request().path().substring(1)).getAbsolutePath());
                handler.response().sendFile(new File(handler.request().path().substring(1)).getAbsolutePath()).end(); 
            } catch(Exception e) {
                e.printStackTrace();
                logger.error("Error: ", e);
            }
        };
    }
}

ai.konduit.serving.WebAppEndpoint

In [80]:
package ai.konduit.serving;

import ai.konduit.serving.endpoint.Endpoint;
import ai.konduit.serving.endpoint.HttpEndpoints;
import ai.konduit.serving.pipeline.api.pipeline.Pipeline;
import ai.konduit.serving.pipeline.api.pipeline.PipelineExecutor;

import java.util.Arrays;
import java.util.List;

public class CustomEndpoints implements HttpEndpoints {
    
    @Override
    public List<Endpoint> endpoints(Pipeline pipeline, PipelineExecutor pipelineExecutor) {
        return Arrays.asList(new WebAppEndpoint(pipelineExecutor));
    }
}

ai.konduit.serving.CustomEndpoints

In [81]:
import java.net.URLClassLoader;
import java.net.URL;
import java.io.File;

import java.util.ArrayList;
import java.util.List;

import org.apache.commons.io.FileUtils;
import java.io.IOException;

import java.nio.charset.StandardCharsets;

URL[] urls = ((URLClassLoader) Class.forName("ai.konduit.serving.vertx.config.InferenceConfiguration").getClassLoader()).getURLs();
List<String> classpaths = new ArrayList<>();

for(URL url : urls) {
    String singleClassPath = new File(url.toURI()).getAbsolutePath();
    System.out.println(singleClassPath);
    classpaths.add(singleClassPath);
}

try {
    String output = String.join(File.pathSeparator, classpaths);
    File classpathOutputPath = new File("classpath");
    FileUtils.writeStringToFile(new File("classpath"), output, StandardCharsets.UTF_8);
    System.out.format("%n-------------%nSaved content:%n-------------%n%s%n\tin file:%n%s%n-------------", output, classpathOutputPath.getAbsolutePath());
} catch (IOException e) {
    e.printStackTrace();
}

/tmp/beaker8984958766066872621/outDir
/root/konduit/konduit.jar

-------------
Saved content:
-------------
/tmp/beaker8984958766066872621/outDir:/root/konduit/konduit.jar
	in file:
/root/konduit/demos/6-bmi-onnx-pytorch/classpath
-------------

null

In [82]:
%%bash
nohup java -cp $(cat classpath) ai.konduit.serving.cli.launcher.KonduitServingLauncher serve -id bmi-onnx-pytorch -c bmi-onnx-pytorch.yaml -rwm &

In [85]:
%%bash
konduit logs bmi-onnx-pytorch -l 300

05:16:58.428 [main] INFO  a.k.s.c.l.command.KonduitRunCommand - Processing configuration: /root/konduit/demos/6-bmi-onnx-pytorch/bmi-onnx-pytorch.yaml
05:16:58.434 [main] INFO  u.o.l.s.context.SysOutOverSLF4J - Replaced standard System.out and System.err PrintStreams with SLF4JPrintStreams
05:16:58.435 [main] INFO  u.o.l.s.context.SysOutOverSLF4J - Redirected System.out and System.err to SLF4J for this context
05:16:58.436 [main] INFO  a.k.s.c.l.command.KonduitRunCommand - Starting konduit server with an id of 'bmi-onnx-pytorch'
05:16:58.759 [vert.x-worker-thread-0] INFO  a.k.s.p.registry.PipelineRegistry - Loaded 28 PipelineStepRunnerFactory instances
05:16:59.121 [vert.x-worker-thread-0] INFO  a.k.serving.python.PythonRunner - Over riding python path :/root/miniconda/lib/python37.zip:/root/miniconda/lib/python3.7:/root/miniconda/lib/python3.7/lib-dynload:/root/miniconda/lib/python3.7/site-packages
05:16:59.711 [vert.x-worker-thread-0] INFO  a.k.serving.python.PythonRunner - Resolving

In [86]:
%%bash
konduit list


Listing konduit servers...

 #   | ID                             | TYPE       | URL                  | PID     | STATUS     
 1   | bmi-onnx-pytorch               | inference  | 0.0.0.0:9009         | 41181   | started    




In [87]:
%%html
<img src="image_me.jpg"/>

In [88]:
%%bash
konduit inspect bmi-onnx-pytorch -q {port}

9009



In [89]:
%%bash
curl -s -H "Content-Type: multipart/form-data" -H "Accept: application/json" -X POST -F "image=@image_me.jpg" http://localhost:9009/predict

{
  "bmi_value" : 22.18,
  "boxes" : [ 447.0, 174.0, 636.0, 470.0 ],
  "predictions" : {
    "@NDArrayType" : "DOUBLE",
    "@NDArrayShape" : [ 8 ],
    "@NDArrayDataBase64" : "AAAAAAAAAAAAAAAAAADwPwAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA=="
  },
  "prob" : 1.0,
  "index" : 1,
  "bmi_class" : "Normal_Range"
}



## Metrics
The cell below also embeds the associated metrics...

In [90]:
%%html

<div style="display: flex; justify-content: center; align-items: center; border: 1px solid black;">
    <iframe src="http://localhost:3000/d/lP_JcnHWz/pipeline-metrics?orgId=1&refresh=5s&kiosk&var-serverName=bmi-onnx-pytorch" width=1500 height=1300>
</div>

## Web application
The cell below demonstrate the web application served by konduit-serving

In [92]:
%%html
<div style="display: flex; justify-content: center; align-items: center; border: 1px solid black;">
    <iframe src="http://localhost:9009/web-app/index.html" allow="camera;microphone", width=1000 height=1000></iframe>
</div>

## View metrics in Browser
Visit: http://localhost:3000/d/lP_JcnHWz/pipeline-metrics?orgId=1&refresh=5s&kiosk&var-serverName=bmi-onnx-pytorch to view metrics.